## 1.- Crear Base de Datos

* Crear con código SQL una base de datos llamada supermercado.
* Código Python con mysql connector que ejecute el SQL para borrar y generar la base de datos vacía

In [1]:
# Ejercicio 1 - Crear base de datos estudiantes
import mysql.connector as con
def crear_bd(sql):
    try:
       connection = con.connect(
            host="localhost",
            port="3306",
            user="root",
            password="ADMIN",
            database='supermercado'
        )
       cursor = connection.cursor()
       cursor.execute(sql)
       
    except con.Error as error:
      print(f"Ha ocurrido un error: {error}") 
      if connection:
         connection.rollback()
         return 0
    finally:
        if cursor: 
            cursor.close()
        if connection:
            connection.close()

In [2]:
sql = "DROP DATABASE IF EXISTS supermercado; CREATE DATABASE IF NOT EXISTS supermercado;"

In [3]:
crear_bd(sql)

# Tablas SQL

* Tiendas

   * id_tienda (PRIMARY KEY)
   * nombre_tienda
   * direccion
   * ciudad


In [4]:
sql = "CREATE TABLE IF NOT EXISTS tiendas (id_tienda INT AUTO_INCREMENT PRIMARY KEY, nombre_tienda VARCHAR(100) NOT NULL, dirección VARCHAR(80), ciudad VARCHAR(50) NOT NULL);"

In [5]:
crear_bd(sql)

* Empleados
    * id_empleado (PRIMARY KEY)
    * nombre_empleado
    * puesto (ej.: Cajero, Gerente, Reponedor)
    * id_tienda (FOREIGN KEY que hace referencia a tiendas.id_tienda)

In [6]:
sql = "CREATE TABLE IF NOT EXISTS empleados (id_empleado INT AUTO_INCREMENT PRIMARY KEY, nombre_empleado VARCHAR(100) NOT NULL, puesto VARCHAR(60), id_tienda INT NOT NULL, FOREIGN KEY (id_tienda) REFERENCES tiendas(id_tienda) ON DELETE CASCADE);"

In [7]:
crear_bd(sql)

* Categorias
    * id_categoria (PRIMARY KEY)
    * nombre_categoria

In [8]:
sql = "CREATE TABLE IF NOT EXISTS categorias (id_categoria INT AUTO_INCREMENT PRIMARY KEY, nombre_categoria VARCHAR(60));"

In [9]:
crear_bd(sql)

* Productos
    * id_producto (PRIMARY KEY)
    * nombre_producto
    * precio
    * stock
    * id_categoria (FOREIGN KEY que hace referencia a categorias.id_categoria)

In [10]:
sql = "CREATE TABLE IF NOT EXISTS productos (id_producto INT AUTO_INCREMENT PRIMARY KEY, nombre_producto VARCHAR(70), precio FLOAT(8,2), stock INT, id_categoria INT NOT NULL, FOREIGN KEY (id_categoria) REFERENCES categorias(id_categoria));"

In [11]:
crear_bd(sql)

* Clientes
    * id_cliente (PRIMARY KEY)
    * first_name
    * last_name
    * email
    * codigo_postal

In [12]:
sql = "CREATE TABLE IF NOT EXISTS clientes (id_cliente INT AUTO_INCREMENT PRIMARY KEY, first_name VARCHAR(50), last_name VARCHAR(50), email VARCHAR(100), codigo_postal INT);"

In [13]:
crear_bd(sql)

* Ordenes
    * id_orden (PRIMARY KEY)
    * id_cliente (FOREIGN KEY que hace referencia a clientes.id_cliente)
    * id_empleado (FOREIGN KEY que hace referencia a empleados.id_empleado)
    * fecha_orden
    * metodo_pago (una enum que solo admita tres valores Tarjeta, Efectivo)

In [14]:
sql = "CREATE TABLE IF NOT EXISTS ordenes (id_orden INT AUTO_INCREMENT PRIMARY KEY, id_cliente INT, id_empleado INT, fecha_orden DATE, metodo_pago ENUM('Tarjeta', 'Efectivo') DEFAULT 'Efectivo', FOREIGN KEY (id_cliente) REFERENCES clientes(id_cliente), FOREIGN KEY (id_empleado) REFERENCES empleados(id_empleado));"

In [15]:
crear_bd(sql)

* Detalle_orden
    * id_detalle (PRIMARY KEY)
    * id_orden (FOREIGN KEY que hace referencia a ordenes.id_orden) NOT NULL
    * id_producto (FOREIGN KEY que hace referencia a productos.id_producto) NOT NULL
    * cantidad
    * precio_unitario: mismo precio que en la tabla producto
    * descuento (podría ser NULL si no se aplica)

In [16]:
sql = "CREATE TABLE IF NOT EXISTS detalle_orden (id_detalle INT AUTO_INCREMENT PRIMARY KEY, id_orden INT NOT NULL, id_producto INT NOT NULL, cantidad FLOAT(8,2), precio_unitario FLOAT(8,2), descuento TINYINT, FOREIGN KEY (id_orden) REFERENCES ordenes(id_orden), FOREIGN KEY (id_producto) REFERENCES productos(id_producto));"

In [17]:
crear_bd(sql)

# Tablas

## 2.- Generar datos demo desde Python

In [18]:
def insertar_dataframe(df, table_name):
    try:
        connection = con.connect(
            host="localhost",
            port="3306",
            user="root",
            password="ADMIN",
            database='supermercado'
        )
        cursor = connection.cursor()
        
        columns = ','.join(df.columns)
        placeholders = ','.join(['%s'] * len(df.columns))
        sql = f'INSERT INTO {table_name} ({columns}) VALUES ({placeholders});'
    
        rows = [tuple(row) for index, row in df.iterrows()] # lista de tuplas con los datos del dataframe

        cursor.executemany(sql, rows)
        
        connection.commit()
        return cursor.rowcount
    except con.Error as error:
        print(f"Ha ocurrido un error: {error}")
        if connection:
            connection.rollback()
        return 0
    finally:
        if cursor: 
            cursor.close()
        if connection:
            connection.close()

* Tabla Tiendas

In [19]:
import pandas as pd
import random
import datetime

tiendas = ['Tienda Centro','Tienda Norte','Tienda Sur','Tienda Poniente','Tienda Este','Tienda Oeste','Central Tiendas']
tiendas

dirección = ['Avd.Centro,nº1','Calle Norte, nº2','Calle Sur, nº3', 'Avd. Poniente, nº4','Calle Este, nº5','Calle Oeste, nº6', 'Crta.Central, Km10']
   
dirección

# Ciudad

ciudad= ['Almería','Girona','Murcia','Barcelona','Orense','Gijón','Madrid' ] 
ciudad 

# Creamos un DataFrame 

df_tiendas = pd.DataFrame(zip(tiendas, dirección, ciudad),
             columns= ['nombre_tienda', 'dirección', 'ciudad']) # El comando Zip empaqueta todas las variante hemos hecho | y con columns damos nombres a las columnas

# Añadir columna Pk:

df_tiendas = df_tiendas.reset_index().rename({'index': 'id_tienda'}, axis=1)
df_tiendas['id_tienda'] = df_tiendas['id_tienda'] + 1
df_tiendas




,id_tienda,nombre_tienda,dirección,ciudad
0,1,Tienda Centro,"Avd.Centro,nº1",Almería
1,2,Tienda Norte,"Calle Norte, nº2",Girona
2,3,Tienda Sur,"Calle Sur, nº3",Murcia
3,4,Tienda Poniente,"Avd. Poniente, nº4",Barcelona
4,5,Tienda Este,"Calle Este, nº5",Orense
5,6,Tienda Oeste,"Calle Oeste, nº6",Gijón
6,7,Central Tiendas,"Crta.Central, Km10",Madrid


In [20]:
insertar_dataframe(df_tiendas, 'tiendas')

7

* Tabla Empleados

In [21]:
# Creamos lista de empleados | Nombre,Apellidos,Función
empleados = list()
nombres = ['Alejandro', 'María', 'Carlos', 'Lucía', 'Fernando', 'Isabel', 'Javier', 'Ana', 'Miguel', 'Sofía', 'José', 'Clara', 'Luis', 'Elena', 'Daniel', 'Marta', 'Andrés', 'Laura', 'Raúl', 'Paula', 'Manuel', 'Sara', 'Pablo', 'Julia', 'Sergio', 'Valeria', 'Tomás', 'Adriana', 'Eduardo', 'Camila', 'Francisco', 'Diana', 'Ricardo', 'Patricia', 'Ángel', 'Carla', 'Diego', 'Irene', 'Samuel', 'Eva', 'David', 'Andrea', 'Hugo', 'Verónica', 'Cristian', 'Gabriela', 'Mario', 'Natalia', 'Jorge', 'Lorena', 'Rubén', 'Carmen']
apellidos = ['García', 'Martínez', 'López', 'Sánchez', 'Pérez', 'González', 'Rodríguez', 'Fernández', 'Hernández', 'Jiménez', 'Ruiz', 'Díaz', 'Moreno', 'Álvarez', 'Romero', 'Torres', 'Vázquez', 'Domínguez', 'Ramos', 'Ramírez', 'Castro', 'Ortiz', 'Rubio', 'Molina', 'Delgado', 'Cruz', 'Flores', 'Navarro', 'Cabrera', 'Campos', 'Peña', 'Aguilar', 'Santos', 'Reyes', 'Méndez', 'Medina', 'Paredes', 'Serrano', 'Cortés', 'Carrillo', 'Ortiz', 'Velázquez', 'Chávez', 'Guzmán', 'Salinas', 'Esquivel', 'Lara', 'Fuentes', 'Montes']
función = ["Cajero", "Reponedor", "Encargado de sección", "Gerente", "Personal de limpieza", "Panadero", "Carnicero", "Pescadero", "Frutero", "Charcutero", "Jefe de turno", "Operador logístico", "Atención al cliente", "Auxiliar de caja", "Encargado de seguridad","Responsable de zona","Jefe de compras","Encargada carnicería","encargado pescadería","Jefe de Limpieza"]
for tienda in df_tiendas['id_tienda']:
    for empleado in range(20):
        nombre_empleado = (random.choice(nombres) + ' ' + random.choice(apellidos))
        puesto = función[empleado]
        id_tienda = tienda
        empleados.append([nombre_empleado, puesto, id_tienda])
        
# Creamos un DataFrame         
df_empleados = pd.DataFrame(empleados, columns=['nombre_empleado', 'puesto', 'id_tienda'])
        
df_empleados = df_empleados.reset_index().rename({'index': 'id_empleado'}, axis=1)
df_empleados['id_empleado'] = df_empleados['id_empleado'] + 1
df_empleados






,id_empleado,nombre_empleado,puesto,id_tienda
0,1,Carla Cabrera,Cajero,1
1,2,Raúl Hernández,Reponedor,1
2,3,Irene Ortiz,Encargado de sección,1
3,4,María Montes,Gerente,1
4,5,Samuel Sánchez,Personal de limpieza,1
...,...,...,...,...
135,136,Eva Peña,Responsable de zona,7
136,137,Rubén Fuentes,Jefe de compras,7
137,138,Luis Montes,Encargada carnicería,7
138,139,Valeria Ruiz,encargado pescadería,7


In [22]:
insertar_dataframe(df_empleados, "empleados")

140

* Tabla Categorías

In [23]:
categorias = ['Lácteos','Pescados','Carnes','Verduras','Frutas','Bollería y Pan','Refrescos','Vinos','Alcohol','Congelados']

df_categorias= pd.DataFrame(categorias,columns= ['nombre_categoria'])

# Añadir columna pk:

df_categorias = df_categorias.reset_index().rename({'index': 'id_categoria'}, axis=1)
df_categorias['id_categoria'] = df_categorias['id_categoria'] + 1
df_categorias



,id_categoria,nombre_categoria
0,1,Lácteos
1,2,Pescados
2,3,Carnes
3,4,Verduras
4,5,Frutas
5,6,Bollería y Pan
6,7,Refrescos
7,8,Vinos
8,9,Alcohol
9,10,Congelados


In [24]:
insertar_dataframe(df_categorias, 'categorias')

10

* Productos

    * id_producto: valores enteros consecutivos (1, 2, 3...).
    * nombre_producto: nombres como “Leche Entera”, “Manzana Roja”, “Refresco de Cola”, etc.
    * precio: valores DECIMAL entre 0.50 y 50.00, por ejemplo.
    * stock: valores INT entre 0 y 500 (o el rango que quieras).
    * id_categoria: debe hacer referencia a las categorías que hayas definido (1, 2, 3, etc.).



In [25]:
# Listado de productos

# generar columna id_producto
nombres = ["Leche", "Yogur", "Queso", "Mantequilla", "Pollo", "Carne de res", "Cerdo", "Chuletas", "Manzana", "Banana", "Naranja", "Uva", "Zanahoria", "Tomate", "Lechuga", "Cebolla", "Agua", "Jugo de naranja", "Refresco", "Cerveza", "Papas fritas", "Chocolate", "Galletas", "Chicles", "Velas", "Platos desechables", "Bolsas de basura", "Pilas", "Detergente", "Jabón líquido", "Esponjas", "Desinfectante", "Champú", "Colonia", "Crema hidratante", "Desodorante", "Tierra para macetas", "Fertilizante", "Semillas", "Regadera"]
precios, lista_categorias, stock = list(), list(), list()
cantidad = len(nombres)
for _ in range(cantidad):
    precios.append(random.uniform(0.50, 10.50))
    stock.append(random.randint(0, 500))
for categ in df_categorias['id_categoria']:
    for i in range(4):
        lista_categorias.append(categ)
lista_categorias        
df_productos = pd.DataFrame(zip(nombres, precios, stock, lista_categorias), columns=['nombre_producto', 'precio', 'stock', 'id_categoria'])    
df_productos = df_productos.reset_index().rename({'index': 'id_producto'}, axis=1)
df_productos['id_producto'] = df_productos['id_producto'] + 1
df_productos

,id_producto,nombre_producto,precio,stock,id_categoria
0,1,Leche,3.551087,288,1
1,2,Yogur,7.660695,53,1
2,3,Queso,7.430396,449,1
3,4,Mantequilla,8.190713,449,1
4,5,Pollo,1.828038,315,2
5,6,Carne de res,10.156211,45,2
6,7,Cerdo,6.515213,247,2
7,8,Chuletas,1.405606,487,2
8,9,Manzana,7.990156,320,3
9,10,Banana,2.103978,58,3


In [26]:
insertar_dataframe(df_productos, 'productos')

40

* Clientes

    * id_cliente: valores enteros consecutivos (1, 2, 3...).
    * nombre_cliente: nombres y apellidos ficticios (p. ej. “Carlos López”, “María Gil”).
    * email: podrías generar correos ficticios (p. ej. “carlos.lopez@test.com”).
    * telefono: número de 9 o 10 dígitos (dependiendo del país).
    * direccion: calles y números ficticios (p. ej. “Av. Siempre Viva 742”).


In [27]:
# generar columna cliente
nombre_cliente, apellido_cliente, email, postal, cliente_completo = list(), list(), list(), list(), list()
calles = ("Av. Libertad", "Calle Mayor", "Paseo de la Reforma", "Calle Real", "Av. de la Constitución", "Calle del Sol", "Calle de la Luna", "Calle los Pinos", "Calle de la Paz", "Calle Olivo", "Calle Cedro", "Calle Nogal", "Av. de las Flores", "Paseo del Prado", "Calle del Río", "Av. Central", "Calle Primavera", "Calle Otoño", "Calle Invierno", "Calle Verano", "Calle Granada", "Calle Sevilla", "Calle Córdoba", "Calle Málaga", "Calle Valencia", "Calle Zaragoza", "Calle Alicante", "Av. de los Ángeles", "Calle del Carmen", "Calle San Miguel", "Calle San Juan", "Calle del Pilar", "Calle Santa Teresa", "Calle San Francisco", "Calle de la Sierra", "Calle del Mar", "Av. del Norte", "Av. del Sur", "Calle del Este", "Calle del Oeste", "Calle Diamante", "Calle Esmeralda", "Calle Rubí", "Calle Zafiro", "Calle Ámbar", "Calle Turquesa", "Calle Topacio", "Calle Perla", "Calle Horizonte", "Calle Amanecer")
for _ in range(2000):
    nom = random.choice(nombres)
    nombre_cliente.append(nom)
    ape = random.choice(apellidos)
    apellido_cliente.append(ape)
    email.append((f'{nom}.{ape}@mail.com'))
    postal.append(random.randint(0000, 11900))
    
df_clientes = pd.DataFrame(zip(nombre_cliente, apellido_cliente, email, postal), columns=['first_name', 'last_name', 'email', 'codigo_postal']) 
df_clientes = df_clientes.reset_index().rename({'index': 'id_cliente'}, axis=1)
df_clientes['id_cliente'] = df_clientes['id_cliente'] + 1
df_clientes

,id_cliente,first_name,last_name,email,codigo_postal
0,1,Zanahoria,Fuentes,Zanahoria.Fuentes@mail.com,76
1,2,Bolsas de basura,Delgado,Bolsas de basura.Delgado@mail.com,548
2,3,Uva,Torres,Uva.Torres@mail.com,3991
3,4,Jugo de naranja,Cabrera,Jugo de naranja.Cabrera@mail.com,11265
4,5,Chocolate,Chávez,Chocolate.Chávez@mail.com,11475
...,...,...,...,...,...
1995,1996,Banana,Pérez,Banana.Pérez@mail.com,7562
1996,1997,Queso,Jiménez,Queso.Jiménez@mail.com,11272
1997,1998,Uva,Reyes,Uva.Reyes@mail.com,786
1998,1999,Champú,Ortiz,Champú.Ortiz@mail.com,5089


In [28]:
insertar_dataframe(df_clientes, 'clientes')

2000

* Orden
    * id_orden: Valores enteros consecutivos
    * id_cliente: Referencias de los Ids existentes en la tabla Clientes
    * id_empleado: Refeferencias de los Ids existentes en la tabla Empleados
    * fecha_orden: Generar fechas aleatorias o secuenciales
    * metodo_pago: Escoger entre tarjeta o efectivo

In [29]:
# generar columna id_orden
from datetime import timedelta


cliente, emplea, fecha, metodo = list(), list(), list(), list()
fechas = [datetime.datetime(year=2024, month=1, day=1) + timedelta(days=numero) for numero in range(395)]
for _ in range(10000):
    cliente.append(random.choice(df_clientes['id_cliente']))
    emplea.append(random.choice(df_empleados['id_empleado']))
    fecha.append(random.choice(fechas))
    metodo.append(random.choices(['Tarjeta', 'Efectivo'], weights=[0.65, 0.35])[0])

fecha.sort()
df_ordenes = pd.DataFrame(zip(cliente, emplea, fecha, metodo), columns=['id_cliente', 'id_empleado', 'fecha_orden', 'metodo_pago'])

df_ordenes.sort_values('fecha_orden', inplace=True)
df_ordenes = df_ordenes.reset_index().rename({'index': 'id_orden'}, axis=1)
df_ordenes['id_orden'] = df_ordenes['id_orden'] + 1
df_ordenes.sort_values('id_orden', inplace=True)
df_ordenes.head(50)

,id_orden,id_cliente,id_empleado,fecha_orden,metodo_pago
0,1,1490,13,2024-01-01,Tarjeta
18,2,1937,54,2024-01-01,Tarjeta
19,3,1769,48,2024-01-01,Tarjeta
20,4,1243,127,2024-01-01,Efectivo
25,5,89,104,2024-01-01,Tarjeta
22,6,303,51,2024-01-01,Tarjeta
23,7,1650,13,2024-01-01,Tarjeta
24,8,359,6,2024-01-01,Efectivo
32,9,1049,122,2024-01-01,Tarjeta
26,10,4,62,2024-01-01,Tarjeta


In [30]:
insertar_dataframe(df_ordenes, 'ordenes')

10000

* Detalles de ordenes
    * id_detalle: Valores enterors consecutivos
    * id_orden: Referencia al Id de alguna orden válida
    * id_producto: Referencia al ID
    * Cantidad: Valores entre 1 y 20 
    * Precio_unitario: Usando en mismo precio está en la tabla productos 
    * Descuento: Valores Decimales bajos

In [31]:
# generar columna id_detalle

orden_completa = list()
productos = df_productos['id_producto']
orden_20 = df_ordenes['id_orden']

for orden in df_ordenes['id_orden']:
    ordenes = orden
    producto = (int(random.choice(productos)))
    cant = random.randint(1, 20)
    valor = float(df_productos['precio'][df_productos['id_producto'] == producto].iloc[0])
    descuento = random.uniform(0, 0.25)
    orden_completa.append([ordenes, producto, cant, round(valor, 2), round(descuento, 2)])   
    
for orden in range(20000):
    ordenes = random.choice(orden_20)
    producto = (int(random.choice(productos)))
    cant = random.randint(1, 20)
    valor = float(df_productos['precio'][df_productos['id_producto'] == producto].iloc[0])
    descuento = random.uniform(0, 0.25)
    orden_completa.append([ordenes, producto, cant, round(valor,2), round(descuento, 2)])   
    
df_detalle_orden = pd.DataFrame(orden_completa, columns=['id_orden', 'id_producto', 'cantidad', 'precio_unitario', 'descuento'])

df_detalle_orden = df_detalle_orden.reset_index().rename({'index': 'id_detalle'}, axis=1)
df_detalle_orden['id_detalle'] = df_detalle_orden['id_detalle'] + 1
df_detalle_orden

,id_detalle,id_orden,id_producto,cantidad,precio_unitario,descuento
0,1,1,6,20,10.16,0.14
1,2,2,39,17,6.74,0.04
2,3,3,40,9,6.88,0.06
3,4,4,2,19,7.66,0.23
4,5,5,37,11,1.90,0.21
...,...,...,...,...,...,...
29995,29996,5387,19,14,2.91,0.19
29996,29997,2480,32,1,9.12,0.08
29997,29998,8039,31,18,2.51,0.15
29998,29999,5139,40,16,6.88,0.23


In [32]:
insertar_dataframe(df_detalle_orden, 'detalle_orden')

30000

## 3.- Consultas SQL

In [33]:
def consultas(sql):
    try:
        connection = con.connect(
            host="localhost",
            port="3306",
            user="root",
            password="ADMIN",
            database='supermercado'
        )
        cursor = connection.cursor()
        cursor.execute(sql)
        return cursor.fetchall()
    except con.Error as error:
        print(f"Ha ocurrido un error: {error}")
        if connection:
            connection.rollback()
        return 0
    finally:
        if cursor: 
            cursor.close()
        if connection:
            connection.close()

* 1.-Listado de órdenes con detalles de cliente y empleado

    * Muestra el ID de la orden, la fecha, el nombre del cliente, el nombre del empleado que atendió la compra y el método de pago.
    * Utiliza un JOIN entre las tablas ordenes, clientes y empleados

In [34]:
sql = ('SELECT o.id_orden, o.fecha_orden, c.first_name, c.last_name, e.nombre_empleado, o.metodo_pago FROM ordenes o JOIN clientes c ON c.id_cliente = o.id_cliente JOIN empleados e ON e.id_empleado = o.id_empleado ORDER BY o.id_orden;') 
consultas(sql)

[(1,
  datetime.date(2024, 1, 1),
  'Esponjas',
  'Cabrera',
  'María Aguilar',
  'Tarjeta'),
 (2,
  datetime.date(2024, 1, 1),
  'Bolsas de basura',
  'Cortés',
  'Andrea Paredes',
  'Tarjeta'),
 (3,
  datetime.date(2024, 1, 1),
  'Papas fritas',
  'Guzmán',
  'Lucía Molina',
  'Tarjeta'),
 (4, datetime.date(2024, 1, 1), 'Yogur', 'Chávez', 'Rubén Castro', 'Efectivo'),
 (5, datetime.date(2024, 1, 1), 'Cerdo', 'Rodríguez', 'Lucía Cruz', 'Tarjeta'),
 (6,
  datetime.date(2024, 1, 1),
  'Pilas',
  'Romero',
  'Ángel Carrillo',
  'Tarjeta'),
 (7, datetime.date(2024, 1, 1), 'Tomate', 'Ramos', 'María Aguilar', 'Tarjeta'),
 (8, datetime.date(2024, 1, 1), 'Uva', 'López', 'David Montes', 'Efectivo'),
 (9,
  datetime.date(2024, 1, 1),
  'Chocolate',
  'Martínez',
  'Pablo Navarro',
  'Tarjeta'),
 (10,
  datetime.date(2024, 1, 1),
  'Jugo de naranja',
  'Cabrera',
  'Clara Navarro',
  'Tarjeta'),
 (11,
  datetime.date(2024, 1, 1),
  'Platos desechables',
  'Cabrera',
  'Carlos Rodríguez',
  'Tarje

* 2.- Productos con stock bajo

    * Filtra aquellos productos cuyo stock sea menor a 10.
    * Muestra nombre del producto, categoría y stock.

In [35]:
sql = ('SELECT p.nombre_producto, c.nombre_categoria, p.stock FROM productos p JOIN categorias c ON p.id_categoria = c.id_categoria WHERE p.stock < 10;')
consultas(sql)

[]

* 3.-Ventas totales por categoría

    * Muestra el nombre de la categoría y la suma total de las ventas (ej.: multiplicando cantidad * precio_unitario) para cada categoría.
    * Realiza el JOIN con detalle_orden, productos y categorias.
    * Utiliza agrupación (GROUP BY).

In [36]:
sql = ('SELECT c.nombre_categoria as categoria, sum((d.precio_unitario * ( 1 - d.descuento)) * d.cantidad) as ventas FROM categorias c JOIN productos p ON p.id_categoria = c.id_categoria JOIN detalle_orden d ON d.id_producto = p.id_producto GROUP BY c.id_categoria;')
consultas(sql)

[('Pescados', 156525.0),
 ('Congelados', 183160.79),
 ('Lácteos', 222861.58),
 ('Bollería y Pan', 136734.8),
 ('Refrescos', 187001.49),
 ('Alcohol', 248998.83),
 ('Verduras', 134193.21),
 ('Vinos', 234887.56),
 ('Carnes', 227171.36),
 ('Frutas', 93073.04)]

* 4.-Clientes con mayores gastos acumulados


    * Muestra el nombre del cliente y el monto total que ha gastado (suma de todas sus órdenes).
    * Asegúrate de tener en cuenta posibles descuentos (descuento) si se ha definido. Por ejemplo, la fórmula podría ser (cantidad * precio_unitario) - descuento.
    * Ordena el resultado de mayor a menor gasto acumulado.

In [37]:
sql = ('SELECT c.first_name as nombre, c.last_name as apellido, sum((d.precio_unitario * ( 1 - d.descuento)) * d.cantidad) as compras FROM clientes c JOIN ordenes o ON c.id_cliente = o.id_cliente JOIN detalle_orden d ON d.id_orden = o.id_orden GROUP BY c.id_cliente ORDER BY sum((d.precio_unitario * ( 1 - d.descuento)) * d.cantidad) DESC;')
consultas(sql)

[('Bolsas de basura', 'Álvarez', 2980.87),
 ('Banana', 'Castro', 2949.37),
 ('Bolsas de basura', 'Cortés', 2836.81),
 ('Jugo de naranja', 'Serrano', 2795.38),
 ('Mantequilla', 'González', 2793.29),
 ('Galletas', 'Flores', 2635.01),
 ('Colonia', 'Romero', 2592.22),
 ('Queso', 'Fernández', 2472.06),
 ('Pollo', 'Moreno', 2463.62),
 ('Cebolla', 'Santos', 2450.85),
 ('Platos desechables', 'Domínguez', 2444.01),
 ('Galletas', 'Navarro', 2438.21),
 ('Queso', 'Lara', 2369.87),
 ('Regadera', 'Ruiz', 2333.44),
 ('Cerdo', 'Santos', 2328.45),
 ('Semillas', 'Torres', 2314.57),
 ('Chuletas', 'Álvarez', 2308.76),
 ('Pollo', 'Fernández', 2298.41),
 ('Lechuga', 'Sánchez', 2294.99),
 ('Naranja', 'Fernández', 2289.09),
 ('Regadera', 'Torres', 2270.53),
 ('Crema hidratante', 'Jiménez', 2264.6),
 ('Chicles', 'García', 2243.29),
 ('Mantequilla', 'Medina', 2221.83),
 ('Bolsas de basura', 'Montes', 2210.59),
 ('Desodorante', 'Lara', 2208.13),
 ('Lechuga', 'Jiménez', 2197.08),
 ('Tierra para macetas', 'Jiménez

* 5.-Empleados y número de órdenes gestionadas

    * Muestra el nombre del empleado, el puesto y la cantidad de órdenes que ha gestionado.
    * Utiliza GROUP BY y COUNT.

In [39]:
sql = ('SELECT e.nombre_empleado AS EMPLEADO, e.puesto AS PUESTO, count(o.id_empleado) AS ORDENES FROM empleados e JOIN ordenes o ON o.id_empleado = e.id_empleado GROUP BY e.id_empleado')
consultas(sql)

[('Carla Cabrera', 'Cajero', 79),
 ('Raúl Hernández', 'Reponedor', 84),
 ('Irene Ortiz', 'Encargado de sección', 72),
 ('María Montes', 'Gerente', 68),
 ('Samuel Sánchez', 'Personal de limpieza', 72),
 ('David Montes', 'Panadero', 76),
 ('Lorena Chávez', 'Carnicero', 76),
 ('José Rodríguez', 'Pescadero', 78),
 ('Clara Álvarez', 'Frutero', 74),
 ('Irene Paredes', 'Charcutero', 82),
 ('Ana Torres', 'Jefe de turno', 63),
 ('Patricia Lara', 'Operador logístico', 57),
 ('María Aguilar', 'Atención al cliente', 72),
 ('Daniel Vázquez', 'Auxiliar de caja', 61),
 ('Natalia Reyes', 'Encargado de seguridad', 89),
 ('Marta Campos', 'Responsable de zona', 74),
 ('Samuel Velázquez', 'Jefe de compras', 72),
 ('Carla Ramos', 'Encargada carnicería', 72),
 ('Miguel Sánchez', 'encargado pescadería', 69),
 ('Eva Esquivel', 'Jefe de Limpieza', 67),
 ('Clara Paredes', 'Cajero', 60),
 ('Camila Navarro', 'Reponedor', 82),
 ('Daniel Velázquez', 'Encargado de sección', 74),
 ('Laura Santos', 'Gerente', 69),
 ('

* 6.-Ordenes filtradas por fecha y tienda

    * Muestra todas las órdenes que se realizaron en un rango de fechas determinado (ej.: del 1 de enero de 2025 al 31 de enero de 2025) y en una tienda específica.
    * Incluye datos de la tienda y del cliente.

In [43]:
sql = ("SELECT t.id_tienda, t.nombre_tienda AS tienda, o.id_orden AS factura, o.fecha_orden AS fecha, concat(c.first_name,' ',c.last_name) AS cliente FROM tiendas t JOIN empleados e ON t.id_tienda = e.id_tienda JOIN ordenes o ON o.id_empleado = e.id_empleado JOIN clientes c ON c.id_cliente = o.id_cliente WHERE T.id_tienda = 1 and o.fecha_orden > '2024-01-31' and o.fecha_orden < '2024-03-01' ORDER BY o.fecha_orden;")
consultas(sql)

[(1, 'Tienda Centro', 810, datetime.date(2024, 2, 1), 'Yogur Méndez'),
 (1, 'Tienda Centro', 805, datetime.date(2024, 2, 1), 'Colonia García'),
 (1, 'Tienda Centro', 809, datetime.date(2024, 2, 1), 'Semillas Romero'),
 (1,
  'Tienda Centro',
  794,
  datetime.date(2024, 2, 1),
  'Jugo de naranja Salinas'),
 (1, 'Tienda Centro', 806, datetime.date(2024, 2, 1), 'Naranja López'),
 (1, 'Tienda Centro', 795, datetime.date(2024, 2, 1), 'Papas fritas Aguilar'),
 (1, 'Tienda Centro', 834, datetime.date(2024, 2, 2), 'Detergente Flores'),
 (1, 'Tienda Centro', 842, datetime.date(2024, 2, 2), 'Carne de res Ortiz'),
 (1, 'Tienda Centro', 822, datetime.date(2024, 2, 2), 'Desodorante Hernández'),
 (1, 'Tienda Centro', 832, datetime.date(2024, 2, 2), 'Semillas Rubio'),
 (1, 'Tienda Centro', 836, datetime.date(2024, 2, 2), 'Detergente Martínez'),
 (1,
  'Tienda Centro',
  867,
  datetime.date(2024, 2, 3),
  'Platos desechables Santos'),
 (1, 'Tienda Centro', 865, datetime.date(2024, 2, 3), 'Desinfecta

* 7.-Ranking de productos más vendidos en cada tienda


    * Para cada tienda, muestra los 3 productos más vendidos (en términos de cantidad total).
    * Tendrás que unir tiendas, empleados, ordenes y detalle_orden, además de productos.
    * Usa GROUP BY y ordena por la cantidad sumada (y opcionalmente, un LIMIT 3).

In [ ]:
		
sql = ('WITH ranking_productos AS (SELECT t.nombre_tienda, p.nombre_producto, SUM(do.cantidad) AS total_vendido, ROW_NUMBER() OVER (PARTITION BY t.id_tienda ORDER BY SUM(do.cantidad) DESC) AS ranking FROM tiendas t JOIN empleados e ON t.id_tienda = e.id_tienda JOIN ordenes o ON e.id_empleado = o.id_empleado JOIN detalle_orden do ON o.id_orden = do.id_orden JOIN productos p ON do.id_producto = p.id_producto GROUP BY t.id_tienda, t.nombre_tienda, p.id_producto, p.nombre_producto) SELECT nombre_tienda, nombre_producto, total_vendido FROM ranking_productos WHERE ranking <= 3 ORDER BY nombre_tienda, ranking;')
consultas(sql)		
			

[('Central Tiendas', 'Mantequilla', 1454.0),
 ('Central Tiendas', 'Chocolate', 1448.0),
 ('Central Tiendas', 'Queso', 1360.0),
 ('Tienda Centro', 'Galletas', 1501.0),
 ('Tienda Centro', 'Champú', 1336.0),
 ('Tienda Centro', 'Pilas', 1320.0),
 ('Tienda Este', 'Pilas', 1340.0),
 ('Tienda Este', 'Manzana', 1334.0),
 ('Tienda Este', 'Champú', 1310.0),
 ('Tienda Norte', 'Queso', 1385.0),
 ('Tienda Norte', 'Lechuga', 1382.0),
 ('Tienda Norte', 'Fertilizante', 1370.0),
 ('Tienda Oeste', 'Banana', 1422.0),
 ('Tienda Oeste', 'Queso', 1359.0),
 ('Tienda Oeste', 'Desodorante', 1336.0),
 ('Tienda Poniente', 'Pilas', 1404.0),
 ('Tienda Poniente', 'Bolsas de basura', 1331.0),
 ('Tienda Poniente', 'Mantequilla', 1312.0),
 ('Tienda Sur', 'Semillas', 1477.0),
 ('Tienda Sur', 'Queso', 1404.0),
 ('Tienda Sur', 'Velas', 1365.0)]